##Pyspark vs SQL
###NO OLVIDAR
- SQL = Declarativo
- PySpark = transformaciones encadenadas desde dataframes

In [0]:
%sql /* MAGIC COMMAND: Comando mágico que le dice a la celda que se ejecute como SQL */ 
SELECT * FROM samples.bakehouse.sales_customers
LIMIT 5

In [0]:
df = spark.read.table("samples.bakehouse.sales_customers")
df_selected = df.select("*").limit(10)

df_limited = df_selected.limit(4)
df_limited.display()

In [0]:
display(df_selected)

In [0]:
%sql
SELECT DISTINCT country
FROM samples.bakehouse.sales_customers

In [0]:
df_distinct = df.select("country").distinct()

display(df_distinct)

In [0]:
%sql
SELECT *
FROM samples.bakehouse.sales_customers
WHERE 1 = 1 /* Es para que siempre sea TRUE la consulta y cambiar otras condiciones solo comentando para seguir probando*/
AND gender = 'female'
AND country = 'Japan'
LIMIT 5


In [0]:
from pyspark.sql.functions import col

df_filtered = df.filter((col("country") == "USA") & (col("gender") == "female"))

df_filtered.display()

## ORDER BY + LIMIT

In [0]:
%sql
SELECT *
FROM samples.bakehouse.sales_customers
ORDER BY first_name ASC
LIMIT 10

In [0]:
df_sort = df.orderBy(col("first_name").desc())

display(df_sort.limit(4))

##CREATE COLUMN

In [0]:
%sql
SELECT concat(first_name, ' ', last_name) as fullName,
concat_ws(' , ',address,city,state,country) as fullAddress,
12 as age,
true AS is_true
FROM samples.bakehouse.sales_customers
LIMIT 10

In [0]:
from pyspark.sql.functions import concat,lit,col

df_plus = df.select(
    concat(col("first_name"), lit(" "), col("last_name")).alias("fullName"),
    lit(True).alias("isTrue"),
    lit("random_coment").alias("comment"),
    lit("Estás aprendiendo").alias("You have learned")
).limit(10)

df_plus.display()

##GROUP BY + AGGREGATION 

In [0]:
%sql
SELECT country, count(*) AS Cantidad
FROM samples.bakehouse.sales_franchises
GROUP BY country
ORDER BY cantidad DESC


In [0]:
df_2 = spark.read.table("samples.bakehouse.sales_franchises")

In [0]:
from pyspark.sql.functions import count

df_grouped = (
    df
    .groupBy("country")
    .agg(count("*").alias("Contador"))
    .orderBy(col("Contador").desc())
)

display(df_grouped)

##JOIN (clave para silver)

In [0]:
%sql
SELECT 
concat(c.first_name," ",c.last_name) as fullName,
c.country,
c.gender,
s.paymentMethod,
s.dateTime
FROM samples.bakehouse.sales_transactions s 
JOIN samples.bakehouse.sales_customers c
  ON s.customerID = c.customerID

In [0]:
df_sales = spark.read.table("samples.bakehouse.sales_transactions")

df_joined = (df
             .join(df_sales, on="customerID", how="inner")
             .select("*")
             )

display(df_joined)

##CASTEO

In [0]:
%sql
SELECT
  CAST(transactionID AS STRING) AS transactionID,
  CAST(customerID AS STRING) AS customerID,
  CAST(franchiseID AS STRING) AS franchiseID,
  CAST(dateTime AS STRING) AS dateTime,
  CAST(product AS STRING) AS product
FROM samples.bakehouse.sales_transactions

In [0]:
df_casted = df_sales.select(
    col("transactionID").cast("long"),
    col("customerID").cast("long"),
    col("franchiseID").cast("long"),
    col("dateTime").cast("timestamp"),
    col("product").cast("string"),
    col("totalPrice").cast("string")
)

df_casted.display()

In [0]:
df_test = df_casted.withColumn("totalPrice", col("totalPrice").cast("bigInt"))

df_test.display()

In [0]:
df_drop = df_test.drop("franchiseID")

df_drop.display()